In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用大型文件進行問答

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/language/use-cases/document-qa/question_answering_documents.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/use-cases/document-qa/question_answering_documents.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/use-cases/document-qa/question_answering_documents.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者| [Lavi Nigam](https://github.com/lavinigam-gcp) |


## 概觀

本筆記本展示如何針對多個大型文件建立一個問答 (Q&A) 系統 (或「機器人」)，以便 Vertex AI PaLM API 能夠回答與這些文件內容相關的任何問題。

許多公司將大量資訊儲存在文件中，但要輕鬆而快速擷取這些資訊可能具有挑戰性。為了解決此問題，你將建立由 PaLM API 提供支援的一個問答系統，讓使用者能夠從這些文件 (可以是任何標準文件格式，例如 .pdf、.doc、.docx、.txt、.pptx 或 .html) 中擷取或查詢重要細節。

針對大型文件建置問答系統的挑戰在於，你必須執行的不只是將整個文件傳遞到提示中，作為提示內容。這是因為 LLM，包括 [Vertex PaLM API](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models)，具有會 [限制你可以提供的內容量](https://ai.google/static/documents/palm2techreport.pdf) 的 Token 限制。

那麼你要如何建置一個受 Token 長度限制的問答系統？為解決此問題，除了你的問題 (你的提示) 之外，你還需要提供相關內容；內容來自你的封閉域來源 (即大型文件)。

在本筆記本中，你將看到可以解決大型內容挑戰的三種方法，如下所示：

* **壓入** - 將整個文件內容作為內容推入。
* **Map-Reduce** - 將文件分成較小的區塊。
* **Map-Reduce - 嵌入** - 建立較小區塊的嵌入，並使用向量相似性搜尋來尋找相關內容。

本筆記本會向你介紹使用 Vertex PaLM API 建置問答機器人並為使用者查詢找出相關內容的基本方法，同時控制內容限制。

此外，還有步驟的開放原始碼或 Google Cloud 直接替換，稍後將在本筆記本中討論。


### 目標

在完成筆記本的學習後，你將會學習如何使用 PaLM API 建立能處理大型檔案的問答系統。

你也會學習兩種方法的概念性實作，以協助你嵌入來自許多檔案的大量內容。

整體而言，本筆記本將涵蓋下列主題：

* 安裝 Vertex AI SDK 和其他依賴
* 對你的筆記本環境進行身分驗證
* 匯入函式庫並載入模型
* 鏈和索引鏈簡介
* 方法 1：填充
* 方法 2：Map Reduce
* 方法 3：有嵌入的 Map Reduce


### 費用

本教學指南使用 Google Cloud 的計費元件：

* Vertex AI 生成式 AI Studio

了解 [Vertex AI 價格](https://cloud.google.com/vertex-ai/pricing)，
並使用 [價格計算器](https://cloud.google.com/products/calculator/)
根據預計使用情況產生費用估算值。


## 開始使用


### 安裝 Vertex AI SDK 和其他依賴項


In [ ]:
# Base system dependencies
!sudo apt -y -qq install tesseract-ocr libtesseract-dev

# required by PyPDF2 for page count and other pdf utilities
!sudo apt-get -y -qq install poppler-utils python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig

# Python dependencies
!pip install google-cloud-aiplatform pytesseract PyPDF2 textract --upgrade --quiet --user

***Colab 獨有** *：取消以下Cell註解以重新啟動核心，或使用按鈕重新啟動核心。對於 Vertex AI Workbench，可以使用頂端的按鈕重新啟動終端機。


In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 驗證筆記本環境
* 如果你使用 **Colab** 執行此筆記本，取消註解下方的Cell並繼續。
* 如果你使用 **Vertex AI 工作台** ，請查看[此處](https://github.com/doggy8088/generative-ai/tree/main/setup-env)的設定說明。


In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### 匯入函式庫


**Colab 專用：** 取消下一個Cell註解，以初始化 Vertex AI SDK。對於 Vertex AI Workbench，你不需要執行這項作業。


In [ ]:
# import vertexai

# PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import glob
import os
import re
import warnings

import numpy as np
import pandas as pd
import textract
from PyPDF2 import PdfReader
from tenacity import retry, stop_after_attempt, wait_random_exponential
from vertexai.language_models import TextEmbeddingModel, TextGenerationModel

warnings.filterwarnings("ignore")

### 載入模型


In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")
embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

為使 PaLM API 呼叫更具韌性且符合 [API 配額](https://cloud.google.com/vertex-ai/docs/quotas)，可以使用一種 [指數遞減](https://zh.wikipedia.org/wiki/%E6%8C%87%E6%95%B0%E9%80%9F%E5%88%87) 機制，持續嘗試 API 以確保呼叫成功且不致過度呼叫 API 或違反配額。

如需將你的 API 配額提高，請參考 [此處](https://cloud.google.com/docs/quota_detail/view_manage#requesting_higher_quota)。

你可以在 [此處](https://tenacity.readthedocs.io/en/latest/api.html) 找到現行方法的 API 指南。


In [ ]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(3))
def text_generation_model_with_backoff(**kwargs):
    return generation_model.predict(**kwargs).text


@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(3))
def embedding_model_with_backoff(text=[]):
    embeddings = embedding_model.get_embeddings(text)
    return [each.values for each in embeddings][0]

## 大文件中的問答

在整個產業中最常用的方法之一是使用 LLM 透過 `chains` 來解決包含大量且多個文件的問答問題。

chain 是 LLM 執行任務所需的一系列步驟。例如，chain 可以從 LLM 閱讀一則文件開始，然後詢問文件中的問題，最後針對問題產生回應。

索引 chain 是一種特殊的 chain，它使用索引來儲存和擷取資訊。索引是一種資料結構，可讓 LLM 快速找出相關資訊以進行指定任務。例如，索引 chain 可以使用一個索引來儲存文件中所有人員的名稱，讓 LLM 能夠快速找到所需資訊，以回答關於這些人員的問題。它還可以儲存文件路徑、名稱、頁碼和其他元資料。

目的是建立所有原始文件的簡要索引，讓 LLM 可以輕鬆搜尋龐大的資訊。索引 chain 有助於問答、摘要和聊天機器人。

基本上，有四種 [索引相關 chains](https://docs.langchain.com/docs/components/chains/index_related_chains)：
* 填充
* Map Reduce
* 改善
* Map-Rerank

在這個範例中，你將看到三種方法：填充、Map Reduce 和具有嵌入式的 Map Reduce。


### 方法 1：塞入

塞入是將資料傳遞給大型語言模型 (LLM) 最簡單的方法。你只需將所有資料結合到單一提示中，再將該提示傳遞給 LLM。此方法有兩個優點：

* 它只呼叫 LLM 一次，這可以提升效能。
* LLM 一次就能存取所有資料，這可以提升產出文字的品質。

不過，塞入有一個主要的缺點：它只適用於少量資料。如果你有一個大型資料集，塞入將無法執行。

在深入探討適用於大型文件問答的可能方法之前，你可探索塞入的主要流程以及它在檔案和脈絡較大的情況下會如何失敗。

以下是塞入的流程：

* **文件載入器：** 從來源載入所需文件到你的儲存空間或本機儲存空間。
* **文件處理：** 透過擷取內容和其他元資料來處理文件。
* **脈絡：** 建立脈絡來傳遞前一步驟中擷取的整個內容。
* **提示工程：** 建立問答提示，它會接收前一步驟建立的脈絡，並新增指示以執行特定任務。
* **Vertex PaLM API：** 最後，使用提示和脈絡呼叫 PaLM API 以取得預期的答案。


#### 文件載入器
你開始從雲端儲存空間拷貝文件，並將它們儲存在專案儲存空間或本地。


In [ ]:
# Copying the files from the GCS bucket to local
!mkdir documents
!gsutil -m cp -r gs://github-repo/documents .

你可以在這裡查看其中一項文件：
https://storage.googleapis.com/github-repo/documents/20230426_alphabet_10Q.pdf


#### 文件處理

當你取得文件時，你須針對下遊使用情形處理這些文件。在處理的階段中，你的目標是讀取文件並將其轉換為下遊邏輯可輕易使用的格式。在讀取時，你應盡可能保留原始文件中的所有後設資料。

在本例中，你會載入不同類型的檔案，例如 .pdf、.txt、.docx 及 .json。每種類型的檔案都有其對應的讀取器，而你可以使用稱為 [textract](https://textract.readthedocs.io/en/stable/) 及 [PyPDF2](https://pypdf2.readthedocs.io/en/3.0.0/) 的簡單開源函式庫來載入這些檔案。你可以儲存各個檔案的檔名、檔案類型、頁數 (僅於 pdf 中顯示) 和內容。

在你將後設資料當作脈絡傳送並在稍後回答查詢時，這些後設資料對於引用資訊來源至關重要。

萃取並處理的後設資料和內容之所以必要的原因在於：
* 在將後設資料當作脈絡傳送時，引用資訊來源。
* 回答與文件相關的查詢。
* 追蹤文件中的變更。
* 辨識重複的文件。
* 整理文件。


In [ ]:
def create_data_packet(file_name, file_type, page_number, file_content):
    """Creating a simple dictionary to store all information (content and metadata)
    extracted from the document"""
    data_packet = {}
    data_packet["file_name"] = file_name
    data_packet["file_type"] = file_type
    data_packet["page_number"] = page_number
    data_packet["content"] = file_content
    return data_packet

In [ ]:
final_data = []


def files(path):
    """
    Function that returns only filenames (and not folder names)
    """
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            yield file


for file_name in files("documents/"):
    path = f"documents/{file_name}"
    _, file_type = os.path.splitext(path)
    if file_type == ".pdf":
        # loading pdf files, with page numbers as metadata.
        reader = PdfReader(path)
        for i, page in enumerate(reader.pages):
            text = page.extract_text()
            if text:
                packet = create_data_packet(
                    file_name, file_type, page_number=int(i + 1), file_content=text
                )

                final_data.append(packet)
    else:
        # loading other file types
        text = textract.process(path).decode("utf-8")
        packet = create_data_packet(
            file_name, file_type, page_number=None, file_content=text
        )
        final_data.append(packet)

在從文件中提取內容和元數據時，你可以將它們儲存在 pandas 資料框中，以便於輕鬆的下遊整合，以引用答案提取的來源。此外，在 pandas 資料框中應用文字分塊處理 (將輸入文字分割成較小的字串以符合 token 限制) 也會有所幫助。


In [ ]:
# converting the data that has been read from GCS to Pandas DataFrame for easy readibility and downstream logic
pdf_data = pd.DataFrame.from_dict(final_data)
pdf_data = pdf_data.sort_values(
    by=["file_name", "page_number"]
)  # sorting the datafram by filename and page_number
pdf_data.reset_index(inplace=True, drop=True)
pdf_data.head()

In [ ]:
# you can check how many different file type you have in our datafrmae.
print("Data has these different file types : \n", pdf_data["file_type"].value_counts())

#### 背景選擇


現在，傳統方法中下一步是在向 PaLM API 提問時傳遞背景。

你不知道哪份文件有幫助，因此你可以繼續使用 `content` 欄中的所有文件文字作為背景。


In [ ]:
# combining all the content of the PDF as single string such that it can be passed as context.
context = "\n".join(str(v) for v in pdf_data["content"].values)
print("The total words in the context: ", len(context))

#### 指令工程


接下來，你可以寫一個簡單的提問提示。然後，你可以預先設定提示，讓它遵循一些基本說明。在提示中，你僅要求它在給定的 `內容` 中找到答案後再回答。

你會動態傳遞內容和問題，以便你可以根據需求和實驗進行更改。


In [ ]:
question = "What is the effect of change in accounting estimate for google in 2020?"
prompt = f"""Answer the question as precise as possible using the provided context. If the answer is
              not contained in the context, say "answer not available in context" \n\n
            Context: \n {context}?\n
            Question: \n {question} \n
            Answer:
          """

#### Vertex PaLM API - 答案擷取與評估


在你提示中，你將許多文字作為內容 (大概所有文件)。

你已經知道 [8192 Token](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models) 的輸入 (提示) Token限制適用於 `text-bison@001` 模型，所以你的 PaLM API 呼叫應當失敗。因為根據 ~8k Token 限制，PaLM 模型預期 ~6k 字詞 (輸入 Token)。然而，你僅作為提示就送出 ~`1531642` 字詞。

提醒你，單一 Token 可能小於字詞。Token 大約是四個字元。因此，100 個 Token 大約等於 60-80 個字詞。

因此，你知道為什麼在你想要對大型文件進行問答時，傳統方法無法運作。


In [ ]:
try:
    print("PaLM Predicted:", generation_model.predict(prompt).text)
except Exception as e:
    print(
        "The code failed since it won't be able to run inference on such a huge context and throws this exception: ",
        e,
    )

不過，如果你將內容限制在 5000 字內，或者限制在低於 PaLM API 的 Token 上限，你仍然可以執行這段程式碼。但由於 5000 字可能無法包含你的背景，你很有可能無法得到預期的答案。


In [ ]:
prompt = f"""Answer the question as precise as possible using the provided context. If the answer is
              not contained in the context, say "answer not available in context" \n\n
            Context: \n {context[:5000]}?\n
            Question: \n {question} \n
            Answer:
          """
print("the words in the prompt: ", len(prompt))
print("PaLM Predicted:", generation_model.predict(prompt).text)

因此，現在我們已經看到，將所有文件的內容填塞在一起，並不是建立問答系統的有效方法。有許多不同的方法可以解決這個限制，但在概觀章節中探討過，你會看到兩個基礎且重要的方法：

* Map-Reduce
* Map-Reduce 含有內嵌：Q&A


### 方法 2：映射簡化


[Map Reduce](https://docs.langchain.com/docs/components/chains/index_related_chains) Chains 是一種採用大型語言模型 (LLM) 來處理大量資料的方法。它先將資料分成較小的區塊，在每個區塊上執行初始提示，然後將初始提示的結果與不同的提示合併。

例如，對於問答功能，你可以在每個區塊上執行初始提示來萃取答案，最後再將各個區塊的答案與不同的提示合併。


這種方法的典型流程如下：

* 你取用 N 個文件作為來源。
* 將文件分成 N 個區塊 (假設每個區塊為 1000 個字) 
* 應該將每個區塊作為問答提示的背景提供。
* 透過使用個別提示來總結所有區塊中的答案。


![Embedding Learning](https://storage.googleapis.com/github-repo/img/reference-architecture%20/map_reduce_flow_new.jpeg)


你可以開始撰寫一個函式 `get_chunks_iter`，它會採用一個字串 `text` 和塊大小作為 `maxlength`。

這個函式的目的是將輸入字串 `text` 分割成 `maxlength` 的大小，這是那個塊中的全部字數，再將所有個別塊 lưu lại 成一個 list，並傳回 `final_chunk` list。


In [ ]:
# The function get_chunks_iter() can be used to split a piece of text into smaller chunks,
# each of which is at most maxlength characters long.
# This can be useful for tasks such as summarization, question answering, and translation.
def get_chunks_iter(text, maxlength):
    """
    Get chunks of text, each of which is at most maxlength characters long.

    Args:
        text: The text to be chunked.
        maxlength: The maximum length of each chunk.

    Returns:
        An iterator over the chunks of text.
    """
    start = 0
    end = 0
    final_chunk = []
    while start + maxlength < len(text) and end != -1:
        end = text.rfind(" ", start, start + maxlength + 1)
        final_chunk.append(text[start:end])
        start = end + 1
    final_chunk.append(text[start:])
    return final_chunk


# function to apply "get_chunks_iter" function on each row of dataframe.
# currently each row here for file_type=pdf is content of each page and for other file_type its the whole document.
def split_text(row):
    chunk_iter = get_chunks_iter(row, chunk_size)
    return chunk_iter

`global` 關鍵字用於將變數宣告為全域變數。這表示可以在程式中的任何範圍存取該變數。`chunk_size` 變數宣告為全域變數，因為它將由程式中的其他函式使用。

`pdf_data_sample` 變數是 `pdf_data` 變數的副本。這樣做是因為 `pdf_data` 變數會由程式中的其他函式修改。透過建立變數的副本，你可以確保原始資料不會被修改。


In [ ]:
global chunk_size
# you can define how many words should be there in a given chunk.
chunk_size = 5000

pdf_data_sample = pdf_data.copy()

In [ ]:
# Remove all non-alphabets and numbers from the data to clean it up.
# This is harsh cleaning. You can define your custom logic for cleansing here.
pdf_data_sample["content"] = pdf_data_sample["content"].apply(
    lambda x: re.sub("[^A-Za-z0-9]+", " ", x)
)

`split_text`函式是一個將字串分割成區塊清單的函式，其中每個區塊都是連續的字元序列。在以下第二行程式碼中,
```
pdf_data_sample = pdf_data_sample.explode("chunks")
```
將chunks欄位展開成個別列。這表示pdf_data_sample資料框中的每一列現在都將代表一個單一區塊的文字。


In [ ]:
# Apply the chunk splitting logic here on each row of content in dataframe.
pdf_data_sample["chunks"] = pdf_data_sample["content"].apply(split_text)
# Now, each row in 'chunks' contains list of all chunks and hence we need to explode them into individual rows.
pdf_data_sample = pdf_data_sample.explode("chunks")

In [ ]:
# Sort and reset index
pdf_data_sample = pdf_data_sample.sort_values(by=["file_name", "page_number"])
pdf_data_sample.reset_index(inplace=True, drop=True)
pdf_data_sample.head()

你可以觀察 `20210203_alphabet_10K.pdf` 檔案中單一頁面是如何分成三個區塊的。

你有三個用相同「 1 」標示的頁面，表示一個頁面已經分成三個子集 (區塊)。這很重要，因為現在你有一個可管理的區塊可以傳送作為脈絡，而不是像之前看到的那樣使用整個文件。

這也會增加資料框中的總列數。


In [ ]:
print("The original dataframe has :", pdf_data.shape[0], " rows without chunking")
print("The chunked dataframe has :", pdf_data_sample.shape[0], " rows with chunking")

現在你可以定義提示，並將每個區塊傳遞為語境。


In [ ]:
# function to pass in the apply function on dataframe to extract answer for specific question on each row.
def get_answer(df):
    prompt = f"""Answer the question as precise as possible using the provided context. If the answer is
                 not contained in the context, say "answer not available in context" \n\n
                  Context: \n {df['chunks']}?\n
                  Question: \n {question} \n
                  Answer:
            """

    pred = text_generation_model_with_backoff(prompt=prompt)
    return pred

In [ ]:
# we can take a small sample of the whole dataframe to avoid making too many calls to the API.
pdf_data_sample_head = pdf_data_sample.head(10)

question = "What is the effect of change in accounting estimate for google in 2020?"
pdf_data_sample_head["predicted_answer"] = pdf_data_sample_head.apply(
    get_answer, axis=1
)
pdf_data_sample_head.head(2)

在個別區塊詢問完問題並回答後將所有答案組合成一個新的脈絡。然後傳送這個新的脈絡給最後的提示。在針對每個區塊使用的提示中，當模型找不到任何答案時你已告知模型回傳「脈絡中沒有答案」。

這將協助你移除模型以「脈絡中沒有答案」回應的區塊。其餘區塊會是新的脈絡。


In [ ]:
context_map_reduce = [
    eachanswer
    for eachanswer in pdf_data_sample_head["predicted_answer"].values
    if eachanswer != "answer not available in context"
]

In [ ]:
prompt = f"""Answer the question as precise as possible using the provided context. If the answer is
              not contained in the context, say "answer not available in context" \n\n
            Context: \n {context_map_reduce}?\n
            Question: \n {question} \n
            Answer:
          """
print("the words in the prompt: ", len(prompt))
print("PaLM Predicted:", generation_model.predict(prompt).text)

現在，我們來看一下這種方法的各種優缺點，以總結你所做的。

**優點：** 

* 分塊提高了精度
* 最有助於提取不同文件級別中的實體。
* 由於塊可以並行處理，因此它可以縮放到比其他方法更大、更多的文件。

**缺點：** 

* 多個 API 呼叫可能既昂貴又耗時。
* 速度慢，因為即使早期找到答案，它也會搜尋所有塊。
* 相互矛盾的答案，可能難以解決。

接下來，我們來探討以下方法，它解決了方法 1 的一些缺點。


### 方法 3：使用嵌入式進行 Map 減少


以前問答的方法效率較低，因為它需要對所有文句區塊呼叫 PaLM API。更有效率的方法是建立區塊的嵌入，然後使用向量數學找到類似的區塊。這樣可以讓你從資料框中的所有區塊中找到答案可能存在的相關語境。

這種方法的典型流程如下：
* 將文件分割成區塊。
* 為每個區塊建立嵌入。
* 將問題轉換為嵌入。
* 在問題和區塊嵌入之間執行餘弦相似度來找到最接近的區塊。
* 將最接近的區塊用作 PaLM API 的語境。

這種方法更有效率，因為它只對相關區塊呼叫 PaLM API。


![Embedding Learning](https://storage.googleapis.com/github-repo/img/reference-architecture%20/map_reduce_embedding.jpeg)



你可以先透過獲取每個片段的嵌入，來開始實現。

這將把每個片段的嵌入 (向量 / 數字表示) 作為一個單獨的欄位添加進去。


In [ ]:
pdf_data_sample_head["embedding"] = pdf_data_sample_head["chunks"].apply(
    lambda x: embedding_model_with_backoff([x])
)
pdf_data_sample_head["embedding"] = pdf_data_sample_head.embedding.apply(np.array)
pdf_data_sample_head.head(2)

現在進入此方法的核心本質。首先，你可以定義一個函式 `get_context_from_question`，它包含：
* 使用者想要詢問的 `問題`，
* `vector_store`：向量資料庫儲存，你已在上個步驟建立，以及，
* `sort_index_value`：該值定義在依據餘弦相似度分數執行排序後會擷取多少個區塊。

該函式將擷取 `有效問題`，建立嵌入，並對你傳遞到向量儲存的所有區塊執行點積 (餘弦相似度)。取得分數之後，你可以遞減順序對結果進行排序，並依據 `sort_index_value` 值擷取區塊作為組合字串。

這將成為你詢問問題的內容。


In [ ]:
def get_dot_product(row):
    return np.dot(row, query_vector)


def get_context_from_question(question, vector_store, sort_index_value=2):
    global query_vector
    query_vector = np.array(embedding_model_with_backoff([question]))
    top_matched = (
        vector_store["embedding"]
        .apply(get_dot_product)
        .sort_values(ascending=False)[:sort_index_value]
        .index
    )
    top_matched_df = vector_store[vector_store.index.isin(top_matched)][
        ["file_name", "page_number", "chunks"]
    ]
    context = " ".join(
        vector_store[vector_store.index.isin(top_matched)]["chunks"].values
    )
    return context, top_matched_df

現在你有一個可以總是你為問題取得的相關背景的自訂函式，你可以用每一個新問題來呼叫它。


In [ ]:
# your question for the documents
question = "What efforts have been taken by Google to safeguard their intellectual property in 2020?"

# get the custom relevant chunks from all the chunks in vector store.
context, top_matched_df = get_context_from_question(
    question,
    vector_store=pdf_data_sample_head,
    sort_index_value=5,  # Top N results to pick from embedding vector search
)
# top 5 data that has been picked by model based on user question. This becomes the context.
top_matched_df

In [ ]:
# Prompt for Q&A which takes the custom context found in last step.
prompt = f""" Answer the question as precise as possible using the provided context. \n\n
            Context: \n {context}?\n
            Question: \n {question} \n
            Answer:
          """

# Call the PaLM API on the prompt.
print("PaLM Predicted:", text_generation_model_with_backoff(prompt=prompt))

正如你所看到的，此方法最好的部分是你不必多次呼叫 API。相反，只需要一次，即可找到答案。


現在，讓我們探討這個方法的各種優缺點來總結你所做的。


*  快速：這很快，因為它不需要在所有區塊上執行 API。


*  數據表可能會進入一個龐大的長度，並且餘弦相似性和基本的數學運算可能會變慢。

**優點：** 


*  可用於在嵌入的龐大資料集上有效地計算複雜運算。
*  可用於學習比傳統的文字袋表示法更有資訊性的字詞和詞組的表示法。
*  可用於改善各種自然語言處理任務的效能，例如文字分類、機器翻譯和問答。


**缺點：** 

*  計算向量相似度可能需要大量的運算。
*  可能對嵌入的選擇很敏感。
